In [1]:
import numpy as np
import pandas as pd
import calendar
import copy
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,MinMaxScaler
import calendar
import datetime
from datetime import datetime
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [2]:
carDf= pd.read_csv('oneHotPreprocess.csv')
carDf.head()

,Make_Accura,Make_BMW,Make_Chevrolet,Make_Dodge,Make_Ferrari,Make_Ford,Make_Honda,Make_Jaguar,Make_Lexus,Make_Mazda,...,NumberOfCars_5 to 8,NumberOfCars_more than 8,BasePolicy_All Perils,BasePolicy_Collision,BasePolicy_Liability,Age,Deductible,DriverRating,DaysDiff,Lable
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0.2625,0.000000,0.000000,0.071795,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0.4250,1.000000,1.000000,0.069231,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0.5875,0.666667,0.666667,0.089744,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0.8125,0.333333,0.333333,0.107692,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0.3375,0.000000,0.000000,0.094872,0


In [3]:
#get the lable from the dataset
carDf.head()
carLable= carDf['Lable']
carDf.drop(['Lable'],inplace=True,axis=1) #drop the lable;

In [4]:
carDateNormalized= carDf

In [5]:
#Divide the data into fraudulent and non-fraudulent data
columnlable= list(carDateNormalized.columns.values)

#create the dataframe for fraudulent and non-fraudulent data
nonFraudulent= pd.DataFrame(columns=columnlable)
nonFraudulentLable= pd.DataFrame(columns=['lable'])

fraudulent= pd.DataFrame(columns=columnlable)
fraudulentLable= pd.DataFrame(columns=['lable'])

print("nonFraudulent:",type(nonFraudulent))
print("carDateNormalized:",type(carDateNormalized))

j= 0
k= 0
for i in range(carDateNormalized.shape[0]):
    if(carLable[i]==0):
        nonFraudulent.loc[j]= carDateNormalized.loc[i]
        nonFraudulentLable.loc[j]= 0.0
        j += 1
    else:
        fraudulent.loc[k]= carDateNormalized.loc[i]
        fraudulentLable.loc[i]= 1.0
        k += 1


nonFraudulent: <class 'pandas.core.frame.DataFrame'>
carDateNormalized: <class 'pandas.core.frame.DataFrame'>


In [6]:
print(type(nonFraudulent))
print(type(fraudulent))

#convert to dataframe
nonFraudulentLable= pd.DataFrame(nonFraudulentLable)
fraudulentLable= pd.DataFrame(fraudulentLable)


<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [7]:
#divide the nonFraudulent into test and train
nonFraudX_train,nonFraudX_test,nonFraudY_train,nonFraudY_test = train_test_split(nonFraudulent,nonFraudulentLable,random_state=3,test_size=0.25)
print('nonFraudxtrain:',type(nonFraudX_train))
print(nonFraudX_test.shape)

#divide the trainset into trainset and validation set
nonFraudX_trainNew,nonFraudX_Valid,nonFraudY_trainNew,nonFraudY_Valid= train_test_split(nonFraudX_train,nonFraudY_train,random_state=3,test_size=0.2)



nonFraudxtrain: <class 'pandas.core.frame.DataFrame'>
(3624, 85)


In [8]:
#convert into numpy
nonFraudX_trainNew= nonFraudX_trainNew.values
nonFraudY_trainNew= nonFraudY_trainNew.values

nonFraudX_test= nonFraudX_test.values
nonFraudY_test= nonFraudY_test.values

fraudFeatures= fraudulent.values
fraudLabel= fraudulentLable.values

nonFraudX_Valid= nonFraudX_Valid.values
nonFraudY_Valid= nonFraudY_Valid.values

In [9]:
#create the class Dataset which returns the data and labels
from torch.utils.data.dataset import Dataset, TensorDataset
class myDataset(Dataset):
    
    def __init__(self,features,lables,transform= None):
        dataTensor= []
        lableTensor= []
        dataSize= features.shape[0]
        
        for data in range(dataSize):
            feature= features[data,:]
            #feature= torch.from_numpy(feature).float()
            feature= torch.Tensor(feature)
            dataTensor.append(feature)
            
            lable= np.asanyarray(lables[data])
            lable= torch.from_numpy(lable).float()
           
            #lable= torch.Tensor(lable)
            #print("lable:",lable)
            #assert(False)
           
            lableTensor.append(lable)
        
        #put everything in features and lables
        self.features= dataTensor
        self.lables= lableTensor
    
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self,index):
        
        feature= self.features[index]
        lable= self.lables[index]
        #print("get_item feature:",feature)
        #print("get_item lable:",lable)
        return feature,lable

In [10]:
#make the  nonfraudulent dataset for train and test loader
myNonFraudulentTrainDataset= myDataset(nonFraudX_trainNew,nonFraudY_trainNew)
myNonFraudulentTestDataset= myDataset(nonFraudX_test,nonFraudY_test)

#make validation loader
myNonFraudulentValidation= myDataset(nonFraudX_Valid,nonFraudY_Valid)

#make the fraudulent dataset for train and test loader
myFraudulentDataset= myDataset(fraudFeatures,fraudLabel)


In [11]:
#make the trainloader and test loader for nonfraudulent dataset.
nonFraudulentTrainLoader= torch.utils.data.DataLoader(myNonFraudulentTrainDataset,batch_size=32,shuffle=True,num_workers=0)

nonFraudulentTestLoader= torch.utils.data.DataLoader(myNonFraudulentTestDataset,batch_size=1,shuffle=True,num_workers=0)

#make the trainloader and test loader for validation
nonFraudulentValidationLoader= torch.utils.data.DataLoader(myNonFraudulentValidation,batch_size=32,shuffle=True,num_workers=0)

#make the trainloader and test loader for nonfraudulent dataset.
fraudulentLoader= torch.utils.data.DataLoader(myFraudulentDataset,batch_size=1,shuffle=True,num_workers=0)




In [12]:
#Network architecture for the base autoencoders
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder,self).__init__()
        self.encoder1= nn.Sequential(
            nn.Linear(85,40),
            nn.Sigmoid())
        
        self.decoder1= nn.Sequential(
            nn.Linear(40,85),
            nn.Sigmoid())
    
    def forward(self,x):
        x= self.encoder1(x)
        x= self.decoder1(x)
        return x;

In [13]:
model= autoencoder()
criterion= nn.MSELoss()
optimizer= torch.optim.SGD(model.parameters(), lr=0.01,momentum=0.9,weight_decay=5e-4)
trainingLoss= []
validationLoss= []

In [14]:
#test the model
def testModel(loader):
    running_loss= 0.0
    for i,(feature,lable) in enumerate(loader):
        inputs= torch.tensor(feature)
        output= model(inputs)
        loss= criterion(output,inputs)
        running_loss += loss.item()
    
    return running_loss/nonFraudX_Valid.shape[0]
        

In [15]:
#train the model
trainingLoss= []
validationLoss= []

for epoch in range(350):
    running_loss= 0.0
    for i, (feature,lable) in enumerate(nonFraudulentTrainLoader):
        
        #gets the inputs
        inputs= torch.tensor(feature)
        lables= torch.tensor(lable)
        lables= lables.type(torch.LongTensor)
        
        # =====================forward====================
        output = model(inputs)
        loss= criterion(output,inputs)
     
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # =======print the statistics
        running_loss += loss.item()
        
    print('Epoch--[%d] --------------------Training loss: %.6f' %(epoch + 1,  running_loss /nonFraudX_trainNew.shape[0]))
    
    #test the model on the validation set.
    valid_loss= testModel(nonFraudulentValidationLoader)
    
    print('Epoch--[%d] --------------------Validation loss: %.6f' %(epoch + 1,  valid_loss))
    #append the loss after each epoch
    trainingLoss.append(running_loss/nonFraudX_trainNew.shape[0])
    validationLoss.append(valid_loss)
    
    

Epoch--[1] --------------------Training loss: 0.005973
Epoch--[1] --------------------Validation loss: 0.004606
Epoch--[2] --------------------Training loss: 0.003892
Epoch--[2] --------------------Validation loss: 0.003375
Epoch--[3] --------------------Training loss: 0.003130
Epoch--[3] --------------------Validation loss: 0.002939
Epoch--[4] --------------------Training loss: 0.002848
Epoch--[4] --------------------Validation loss: 0.002763
Epoch--[5] --------------------Training loss: 0.002725
Epoch--[5] --------------------Validation loss: 0.002677
Epoch--[6] --------------------Training loss: 0.002661
Epoch--[6] --------------------Validation loss: 0.002629
Epoch--[7] --------------------Training loss: 0.002623
Epoch--[7] --------------------Validation loss: 0.002600
Epoch--[8] --------------------Training loss: 0.002599
Epoch--[8] --------------------Validation loss: 0.002580
Epoch--[9] --------------------Training loss: 0.002582
Epoch--[9] --------------------Validation loss: 0

Epoch--[73] --------------------Validation loss: 0.002498
Epoch--[74] --------------------Training loss: 0.002508
Epoch--[74] --------------------Validation loss: 0.002498
Epoch--[75] --------------------Training loss: 0.002508
Epoch--[75] --------------------Validation loss: 0.002498
Epoch--[76] --------------------Training loss: 0.002508
Epoch--[76] --------------------Validation loss: 0.002498
Epoch--[77] --------------------Training loss: 0.002508
Epoch--[77] --------------------Validation loss: 0.002497
Epoch--[78] --------------------Training loss: 0.002508
Epoch--[78] --------------------Validation loss: 0.002497
Epoch--[79] --------------------Training loss: 0.002507
Epoch--[79] --------------------Validation loss: 0.002497
Epoch--[80] --------------------Training loss: 0.002507
Epoch--[80] --------------------Validation loss: 0.002497
Epoch--[81] --------------------Training loss: 0.002507
Epoch--[81] --------------------Validation loss: 0.002497
Epoch--[82] ------------------

Epoch--[145] --------------------Training loss: 0.002473
Epoch--[145] --------------------Validation loss: 0.002462
Epoch--[146] --------------------Training loss: 0.002472
Epoch--[146] --------------------Validation loss: 0.002461
Epoch--[147] --------------------Training loss: 0.002471
Epoch--[147] --------------------Validation loss: 0.002460
Epoch--[148] --------------------Training loss: 0.002470
Epoch--[148] --------------------Validation loss: 0.002459
Epoch--[149] --------------------Training loss: 0.002469
Epoch--[149] --------------------Validation loss: 0.002458
Epoch--[150] --------------------Training loss: 0.002468
Epoch--[150] --------------------Validation loss: 0.002457
Epoch--[151] --------------------Training loss: 0.002467
Epoch--[151] --------------------Validation loss: 0.002456
Epoch--[152] --------------------Training loss: 0.002466
Epoch--[152] --------------------Validation loss: 0.002454
Epoch--[153] --------------------Training loss: 0.002464
Epoch--[153] --

Epoch--[216] --------------------Training loss: 0.002374
Epoch--[216] --------------------Validation loss: 0.002362
Epoch--[217] --------------------Training loss: 0.002373
Epoch--[217] --------------------Validation loss: 0.002361
Epoch--[218] --------------------Training loss: 0.002371
Epoch--[218] --------------------Validation loss: 0.002360
Epoch--[219] --------------------Training loss: 0.002370
Epoch--[219] --------------------Validation loss: 0.002359
Epoch--[220] --------------------Training loss: 0.002369
Epoch--[220] --------------------Validation loss: 0.002358
Epoch--[221] --------------------Training loss: 0.002368
Epoch--[221] --------------------Validation loss: 0.002357
Epoch--[222] --------------------Training loss: 0.002367
Epoch--[222] --------------------Validation loss: 0.002356
Epoch--[223] --------------------Training loss: 0.002366
Epoch--[223] --------------------Validation loss: 0.002355
Epoch--[224] --------------------Training loss: 0.002365
Epoch--[224] --

Epoch--[287] --------------------Training loss: 0.002330
Epoch--[287] --------------------Validation loss: 0.002319
Epoch--[288] --------------------Training loss: 0.002329
Epoch--[288] --------------------Validation loss: 0.002318
Epoch--[289] --------------------Training loss: 0.002329
Epoch--[289] --------------------Validation loss: 0.002318
Epoch--[290] --------------------Training loss: 0.002329
Epoch--[290] --------------------Validation loss: 0.002318
Epoch--[291] --------------------Training loss: 0.002329
Epoch--[291] --------------------Validation loss: 0.002318
Epoch--[292] --------------------Training loss: 0.002328
Epoch--[292] --------------------Validation loss: 0.002318
Epoch--[293] --------------------Training loss: 0.002328
Epoch--[293] --------------------Validation loss: 0.002317
Epoch--[294] --------------------Training loss: 0.002328
Epoch--[294] --------------------Validation loss: 0.002317
Epoch--[295] --------------------Training loss: 0.002328
Epoch--[295] --

In [ ]:
trainingLoss= pd.DataFrame(trainingLoss)
validationLoss= pd.DataFrame(validationLoss)

#save the dataframe
trainingLoss.to_csv('Batch8TrainingLoss.csv',index=None,sep='\t')
validationLoss.to_csv('Batch8ValidationLoss.csv',index=None,sep='\t')

In [ ]:
#function for testing the model
def testModel2(loader):
    predictedLabel= []
    lossList= []
    trueLable= []
    for i,(feature,lable) in enumerate(loader):
        inputs= torch.tensor(feature)
        output= model(inputs)
        
        trueLable.append(lable)
        
        loss= criterion(output,inputs)
        lossList.append(loss.item())
        
        if(loss.item()<0.01):
            predictedLabel.append(0)
        else:
            predictedLabel.append(1)
    return predictedLabel,lossList,trueLable


In [ ]:
#test the model on test set
predictedLable1,lossList1,trueLable1= testModel2(myNonFraudulentTestDataset)



In [ ]:
#count the predicted lables
class1= predictedLable1.count(1)
class0= predictedLable1.count(0)
print(class1)
print(class0)

In [ ]:
#test the model on test set
predictedLable2,lossList2,trueLable2= testModel2(myFraudulentTestDataset)


In [ ]:
#count the predicted lables
class1= predictedLable2.count(1)
class0= predictedLable2.count(0)
print(class1)
print(class0)

In [ ]:
#convert list to csv
nonFraudLoss= pd.DataFrame(lossList1)
fraudLoss= pd.DataFrame(lossList2)

In [ ]:
nonFraudLoss.head()

In [ ]:
#save the dataLoss:
nonFraudLoss.to_csv('nonFraudLoss.csv',index=None,sep='\t')
fraudLoss.to_csv('fraudLoss.csv',index=None,sep='\t')

In [ ]:
totalLength1= len(trueLable1)
TP=FP=FN=TN= 0
for i in range(totalLength1):
    if(int(trueLable1[i])==1 and predictedLable1[i]==1):
        TP += 1
    elif(int(trueLable1[i])==1 and predictedLable1[i]==0):
        FN += 1
    elif(int(trueLable1[i])==0 and predictedLable1[i]==0):
        TN += 1
    elif(int(trueLable1[i])==0 and predictedLable1[i]==1):
        FP += 1

totalLength2= len(trueLable2)
for i in range(totalLength2):
    if(int(trueLable2[i])==1 and predictedLable2[i]==1):
        TP += 1
    elif(int(trueLable2[i])==1 and predictedLable2[i]==0):
        FN += 1
    elif(int(trueLable2[i])==0 and predictedLable2[i]==0):
        TN += 1
    elif(int(trueLable2[i])==0 and predictedLable2[i]==1):
        FP += 1
        
print("\nResult ")
print("Accuracy:",(TP+TN)/(TP+FP+FN+TN)*100)
print("Sensitivity:",TP/(TP+FN)*100)
print("Specificity:",TN/(TN+FP)*100)
print("\n")
print("\nTP:",TP)
print("\nFN:",FN)
print("\nFP:",FP)
print("\nTN:",TN)
